In [21]:
import pandas as pd
import re

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
import eli5

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

2022-08-26 22:05:32.243803: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-26 22:05:32.243826: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/annavlasova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import json

with open('../data/full_docs.json', 'r+') as fp:
    full_docs1 = json.load(fp)

In [15]:
df_list = []
for i in range(len(files)):
    df_list.append(pd.DataFrame(full_docs1[files[i]]))

In [16]:
df = pd.concat(df_list)

In [17]:
df['ref_text'] = df['text'].apply(lambda x: re.sub('[\W\d]+', ' ', x.lower()))

In [18]:
df

,data,text,target,ref_text
0,"1. {2}Настоящие Правила устанавливают цели, по...","Настоящие Правила устанавливают цели, порядок ...",2.0,настоящие правила устанавливают цели порядок и...
1,2. {2}Субсидия предоставляется в целях обеспеч...,Субсидия предоставляется в целях обеспечения у...,2.0,субсидия предоставляется в целях обеспечения у...
2,{4}Субсидия предоставляется в рамках государст...,Субсидия предоставляется в рамках государствен...,4.0,субсидия предоставляется в рамках государствен...
3,"3. {5}Получателем бюджетных средств, до которо...","Получателем бюджетных средств, до которого в с...",5.0,получателем бюджетных средств до которого в со...
4,4. {3}Субсидия предоставляется в соответствии ...,Субсидия предоставляется в соответствии с согл...,3.0,субсидия предоставляется в соответствии с согл...
...,...,...,...,...
218,16. Правительства Российской Федерации от 27 ...,16. Правительства Российской Федерации от 27 ...,0.0,правительства российской федерации от апреля ...
219,"17. и изменений, которые вносятся в акты Пра...","17. и изменений, которые вносятся в акты Пра...",0.0,и изменений которые вносятся в акты правитель...
220,18. Правительства Российской Федерации от 24 ...,18. Правительства Российской Федерации от 24 ...,0.0,правительства российской федерации от марта г...
221,19. Правительства Российской Федерации от 29 ...,19. Правительства Российской Федерации от 29 ...,0.0,правительства российской федерации от мая г n...


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['ref_text'], df['target'], 
                                                    test_size=0.33, random_state=42)

In [24]:
text_transformer = TfidfVectorizer(stop_words=russian_stopwords, 
                                   ngram_range=(1, 2), lowercase=True, max_features=150000)

In [25]:
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)

In [26]:
logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)

In [27]:
logit.fit(X_train_text, y_train)

LogisticRegression(C=50.0, multi_class='multinomial', n_jobs=4, random_state=17)

In [28]:
eli5.show_weights(estimator=logit, 
                  feature_names= list(text_transformer.get_feature_names()),
                 top=(50, 5))

/home/annavlasova/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [29]:
test_preds = logit.predict(X_test_text)

In [32]:
train_preds = logit.predict(X_train_text)

In [30]:
from sklearn.metrics import accuracy_score, classification_report

accuracy_score(y_test, test_preds)

0.6939622061760639

In [33]:
accuracy_score(y_train, train_preds)

0.9608777903897087

In [31]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90      1265
         1.0       0.78      0.73      0.75       199
         2.0       0.71      0.72      0.71       172
         3.0       0.49      0.47      0.48        68
         4.0       0.55      0.39      0.46        80
         5.0       0.17      0.04      0.07        23
         6.0       0.39      0.27      0.32        45
         7.0       0.64      0.65      0.64        75
         8.0       0.69      0.46      0.55        24
         9.0       0.00      0.00      0.00        13
        10.0       0.71      0.74      0.72       362
        11.0       0.77      0.75      0.76       396
        12.0       0.41      0.35      0.38       103
        13.0       0.46      0.30      0.36        73
        14.0       0.42      0.33      0.37        15
        15.0       0.52      0.46      0.49       231
        16.0       0.54      0.58      0.56        65
        17.0       0.65    

In [34]:
print(classification_report(y_train, train_preds))

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      2553
         1.0       0.99      1.00      1.00       450
         2.0       0.99      0.99      0.99       341
         3.0       0.92      0.83      0.87       134
         4.0       0.97      0.91      0.94       162
         5.0       0.95      0.92      0.94        39
         6.0       0.92      0.97      0.95       113
         7.0       0.93      0.94      0.94       167
         8.0       0.93      0.93      0.93        40
         9.0       1.00      1.00      1.00        13
        10.0       0.96      0.96      0.96       730
        11.0       0.94      0.98      0.96       811
        12.0       0.99      0.94      0.96       279
        13.0       0.97      0.86      0.92       170
        14.0       0.91      1.00      0.95        21
        15.0       0.93      0.95      0.94       438
        16.0       0.93      0.96      0.95       138
        17.0       0.96    